This script downloads and formats the CNEOS data from the api into a format that blender can handle.

In [1]:
!curl "https://ssd-api.jpl.nasa.gov/fireball.api?req-vel=true&req-vel-comp=true&vel-comp=true&req-loc=true&req-alt=true" > ./fireball.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27655    0 27655    0     0  23490      0 --:--:--  0:00:01 --:--:-- 23496


In [2]:
import json
import pandas as pd

In [3]:
with open('fireball.json') as fh:
    data = json.load(fh)

In [4]:
data['count']

'292'

In [5]:
data['signature']

{'source': 'NASA/JPL Fireball Data API', 'version': '1.0'}

In [6]:
data['fields']

['date',
 'energy',
 'impact-e',
 'lat',
 'lat-dir',
 'lon',
 'lon-dir',
 'alt',
 'vel',
 'vx',
 'vy',
 'vz']

In [7]:
data['data'][0]

['2024-02-26 11:21:01',
 '3.7',
 '0.13',
 '22.9',
 'N',
 '57.1',
 'E',
 '33.1',
 '31.3',
 '-29.3',
 '5.1',
 '9.7']

In [9]:
from datetime import datetime

Hi guys, unfortunatley, I have to work on algo or I will not finish my problem sets in time. So I need you to figure out a few things.

Firsly, the format blender needs can't have certain data types such as strings, and converting ffrom latitude/longitude to polar coordinates to cartesian coordinates would be a pain in the ass to do in blender so we're doing it once here. Just get it to polar coordinates as polar to cartesian is an easy operation in blender.

Next, take all these lists and shove them into a pandas data frame using a dictionary, and then use pandas to write that to a csv (pandas has a function for this but if forget). also, I haven't figured out how velocity. is stored here so you will have to figure that out. it would be best to seperate the components of velocity into their own columns, vx, vy, vz.

Thanks guys!

In [15]:
for date, energy, impact, lat, latdir, lon, londir, alt, vel, vx, vy, vz in data['data']:
    print(
        # datetime.strptime(date, "%Y-%m-%d %H:%M:%S"),
        float(energy),
        float(impact),
        float(lat),
        latdir,
        float(lon),
        londir,
        alt,
        vel,
        vx,
        vy,
        vz
    )
    break

3.7 0.13 22.9 N 57.1 E 33.1 31.3 -29.3 5.1 9.7


In [23]:
!pip install utm
import utm
import math


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [48]:
dates = []
energies = []
impacte = []
Xs = [] # convert latitude, longitude, elevation into polar coordinates
Ys = []
Zs = []
altitudes = []
velocities = []
vxs = []
vys = []
vzs = []
for date, energy, impact, lat, latdir, lon, londir, alt, vel, vx, vy, vz in data['data']:
        dates.append(datetime.strptime(date, "%Y-%m-%d %H:%M:%S"))
        energies.append(float(energy))
        impacte.append(float(impact))
        latitude = math.radians(float(lat))
        longitude = math.radians(float(lon))
        if latdir.lower()=='s':
            latitude *=-1
        if londir.lower()=='w':
            longitude *=-1
        # x, y, _, _ = utm.from_latlon(latitude, longitude)
        x = math.sin(math.pi/2-latitude) * math.cos(longitude)
        y = math.sin(math.pi/2-latitude) * math.sin(longitude)
        z = math.cos(math.pi/2-latitude)
        Xs.append(x)
        Ys.append(y)
        Zs.append(z)
        # latitudes.append(float(lat))
        # longitudes.append(float(lon))
        altitudes.append(float(alt))
        velocities.append(float(vel))
        vxs.append(float(vx))
        vys.append(float(vy))
        vzs.append(float(vz))

In [49]:
fireballs = {
    # 'date': dates, # blender does not parse dates well... could convert to some scalar for mapping to color or time etc.
    'energy': energies,
    'impacte': impacte,
    'X': Xs,
    'Y': Ys,
    'Z': Zs,
    'altitude': altitudes,
    'velocity': velocities,
    'vx': vxs,
    'vy': vys,
    'vz': vzs
}

In [50]:
df = pd.DataFrame(fireballs)
df.to_csv('fireballs.csv', index=False)

In [51]:
pd.read_csv('fireballs.csv')

,energy,impacte,X,Y,Z,altitude,velocity,vx,vy,vz
0,3.7,0.130,0.500364,0.773446,0.389124,33.1,31.3,-29.3,5.1,9.7
1,4.6,0.150,-0.538828,0.807939,-0.238533,23.0,14.4,14.1,-2.7,1.6
2,21.7,0.600,0.078522,0.440820,-0.894154,37.0,28.3,20.7,14.6,12.7
3,3.8,0.130,0.941427,0.295025,-0.163326,37.0,18.3,-17.9,-3.9,0.1
4,2.6,0.092,0.520691,-0.515267,-0.680721,33.8,17.6,2.8,12.7,11.9
...,...,...,...,...,...,...,...,...,...,...
287,62.4,1.500,0.083500,0.710805,-0.698415,29.6,16.3,8.5,-12.1,-6.8
288,43.3,1.100,-0.934403,0.289246,-0.207912,35.0,25.6,10.5,-23.2,-2.9
289,51.8,1.300,-0.310726,0.297975,-0.902585,23.0,20.1,14.8,-8.7,10.4
290,215.0,4.600,0.055367,0.931937,0.358368,26.0,18.2,-1.0,-5.4,-17.3


In [54]:
max(impacte), max(energies)

(440.0, 37500.0)